In [1]:
import numpy as np
import matplotlib.pyplot as plt
from world import Patch, Agent

In [2]:
# Step 1a: Specify a world with P patch types and N patches
P = 5  # Number of patch types
N = 10  # Number of patches
patch_types = np.random.choice(P, N, replace=True)
initial_yields = np.random.rand(P) * 10  # Initial yield for each patch type
decay_rate = 0.1  # Decay rate for all patches

In [11]:
# Step 1a: Create patches based on the world specification
patches = [Patch(initial_yields[patch_type], decay_rate) for patch_type in patch_types]
agent = Agent(beta=0.7, intercept = 0)  # Softmax parameter

In [7]:
leave_times = []
beta = 0.5
intercept = 2
i = 10 

# Run simulation
total_rewards = []
for patch in patches:
    print(patch)
    patch.start_harvesting()  # Start harvesting on the current patch
    rewards = []
    time = 0
    while True:
        reward = patch.get_reward()
        # rewards.append(reward)
        leave_proba = 1 / (1 + np.exp(intercept + beta * reward))
        print(reward, leave_proba)
        i-=1
        if i < 0 :
            break

In [12]:
# Initialize variables to track performance
leave_times = []

# Run simulation
total_rewards = []
for patch in patches:
    patch.start_harvesting()  # Start harvesting on the current patch
    rewards = []
    time = 0
    while True:
        reward = patch.get_reward()
        # rewards.append(reward)
        if agent.choose_action(reward) == 1:  # Decision to leave
            break
        time += 1
    leave_times.append(time)
    # total_rewards.append(sum(rewards))
    patch.harvesting = False  # Stop harvesting when leaving the patch

# Calculate mean and variance of leaving times
E_leave = np.mean(leave_times)
VAR_leave = np.var(leave_times)

print(f"Mean Leaving Time: {E_leave}")
print(f"Variance in Leaving Time: {VAR_leave}")

Mean Leaving Time: 1.6
Variance in Leaving Time: 2.6400000000000006


In [13]:
leave_times

[1, 2, 1, 0, 2, 1, 4, 5, 0, 0]

In [15]:
# Step 2: Run Agents on the World Model

# Step 2a: Compute normative expected leaving time for each patch (assuming MVT)
def compute_mvt_leave_time(patch, background_reward_rate):
    time = 0
    while True:
        reward_rate = patch.initial_yield * np.exp(-patch.decay_rate * time)
        if reward_rate < background_reward_rate:
            break
        time += 1
    return time

background_reward_rate = np.mean(initial_yields) * decay_rate  # Simplified background reward rate

In [19]:
stay_times = [compute_mvt_leave_time(Patch(yield_, decay_rate),
                                    background_reward_rate) for yield_ in initial_yields]
stay_times

[0, 9, 2, 30, 34]

### Patch effect: Increasing yield

In [21]:
increased_yields = np.linspace(1, 20, P)
inc_background_reward_rate = np.mean(increased_yields) * decay_rate
longer_stay_times = [compute_mvt_leave_time(Patch(yield_, decay_rate), 
                                            inc_background_reward_rate) for yield_ in increased_yields]
longer_stay_times

[0, 18, 24, 27, 30]